In [2]:
import numpy as np
import pandas as pd

from star import star_vars
from itertools import combinations

In [3]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if x.shape[0] > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

In [4]:
# helper functions
def apply_unique(func, df, axis=1, *args, **kwargs):
    '''Apply a function to unique rows of a DataFrame
    for efficiency.'''

    applied_df = df.merge(df.drop_duplicates()
                         .assign(**{func.__name__: lambda x: x.apply(func, axis=axis)}), 
                         how='left')
    applied_df.index = df.index
    
    return applied_df
    
    
def scale(df, bounds, axis=1, *args, **kwargs):
    '''scale the sampled matrix
    bounds is a dict with ['ub', 'lb'] keys
    the values are lists of the upper and lower bounds
    of the parameters/variables/factors'''
    
    # numpy equivalent for math operations
    bounds_np = {key:np.array(value) for key,value in bounds.items()}
    
    if axis:
        return df * (bounds_np['ub'] - bounds_np['lb']) + bounds_np['lb']
    else:
        return df.T * (bounds_np['ub'] - bounds_np['lb']) + bounds_np['lb']
    
    
def pairs_h(iterable):
    '''gives the pairs of numbers considering their differences'''
    interval = range(min(iterable), max(iterable)-min(iterable))
    pairs  = {key+1:[j for j in combinations(iterable, 2) if np.abs(j[0]-j[1])==key+1] for key in interval}
    return pairs
    
    
def section_df(df):
    '''gets the paired values of each section based on index'''
    pairs = pairs_h(df.index.get_level_values(-1))
    df_values = df.to_numpy()
    sample = pd.concat({h:
                    pd.DataFrame.from_dict({str(idx_tup): [df_values[idx_tup[0]], df_values[idx_tup[1]]] for idx_tup in idx}, 'index') \
                      for h, idx in pairs.items()})

    return sample
    
    
# lambda functions
'''covariogram of each section'''
cov_section = lambda pair_cols, mu_star: (pair_cols.sub(mu_star, axis=0)[0] * pair_cols.sub(mu_star, axis=0)[1]).groupby(level=[0,1,2]).mean()

'''variogram over all sections'''
variogram = lambda pair_cols: 0.5*(pair_cols[0] - pair_cols[1]).pow(2).groupby(level=[1,2]).mean()

'''morris sensitivity measure equivalent evaluated over all sections'''
morris_eq = lambda pair_cols: ((pair_cols[1] - pair_cols[0]).abs().groupby(level=[1,2]).mean(), \
                               (pair_cols[1] - pair_cols[0]).groupby(level=[1,2]).mean())

'''covariogram over all sections'''
covariogram = lambda pair_cols, mu_overall: ((pair_cols - mu_overall)[0] * (pair_cols - mu_overall)[1]).groupby(level=[1,2]).mean()

'''expected covariogram over all sections'''
e_covariogram = lambda cov_section_all: cov_section_all.groupby(level=[1,2]).mean()

'''sobol (total order) sensitivity measure equivalent evaluated over all sections'''
sobol_eq = lambda gamma, ecov, variance: ((gamma + ecov) / variance).loc[:,1]

# ivars function
def ivars(variogram_array, scale, delta_h):
    '''generate Integrated Variogram Across a Range of Scales (IVARS)
    by approximating area using right trapezoids having width of `delta_h`
    and hights of variogram values'''
    num_h  = len(variogram_value.index.levels[-1].to_list())
    x_bench= np.arange(start=0, stop=delta_h*(num_h+1), step=delta_h)
    x_int  = np.arange(start=0, stop=(scale*10+1)/10, step=delta_h)

    # calculate interpolated values for both x (h) and y (variogram)
    if x_int[-1] < scale:
        x_int.append(scale)
    y_bench= [0] + variogram_array.to_list()

    y_int  = np.interp(x=x_int, xp=x_bench, fp=y_bench)
    
    # for loop for each step size to caluclate the area
    ivars = 0
    for i in range(len(x_int)-1):
        ivars += 0.5*(y_int[i+1] + y_int[i]) * (x_int[i+1] - x_int[i])

    return ivars

# alias
idx = pd.IndexSlice

In [5]:
delta_h = 0.1
rng = np.random.default_rng(seed=100)
star_centres = rng.random((10, 3))
star_points  = star_vars(star_centres, delta_h=delta_h, parameters=['x1', 'x2', 'x3'], rettype='DataFrame')

In [34]:
# bounds = {'ub':[3,4,5], 'lb':[-3,-4,-5]}
# star_points_scaled = scale(star_points, bounds)
rng.random((10,3))

array([[0.61310886, 0.61343421, 0.68586212],
       [0.90635007, 0.30931227, 0.40063662],
       [0.31583786, 0.72192806, 0.03406086],
       [0.40358433, 0.40453942, 0.39342567],
       [0.56853733, 0.24543   , 0.89184929],
       [0.37814582, 0.95769788, 0.93040383],
       [0.34768689, 0.36878674, 0.61965667],
       [0.39690697, 0.10778987, 0.2820133 ],
       [0.18593984, 0.42113953, 0.8903008 ],
       [0.46804122, 0.04604913, 0.76776746]])

In [33]:
# df = apply_unique(ishigami, star_points_scaled, axis=1)
df = apply_unique(ishigami, star_points, axis=1)
df.index.names=['centre', 'param', 'points']
df

0         1         2  ishigami
centre param points                                        
0      x1    0       0.034982  0.596554  0.288863  2.244282
             1       0.134982  0.596554  0.288863  2.343915
             2       0.234982  0.596554  0.288863  2.442202
             3       0.334982  0.596554  0.288863  2.538162
             4       0.434982  0.596554  0.288863  2.630836
...                       ...       ...       ...       ...
9      x3    5       0.408518  0.389765  0.531648  1.409483
             6       0.408518  0.389765  0.631648  1.411058
             7       0.408518  0.389765  0.731648  1.413588
             8       0.408518  0.389765  0.831648  1.417398
             9       0.408518  0.389765  0.931648  1.422860

[300 rows x 4 columns]

In [8]:
# getting the paired values of each section based on `h`
pair_df = df[ishigami.__name__].groupby(level=[0,1]).apply(section_df)
pair_df.index.names = ['centre', 'param', 'h', 'pair_ind']
pair_df

0         1
centre param h pair_ind                    
0      x1    1 (0, 1)    2.244282  2.343915
               (1, 2)    2.343915  2.442202
               (2, 3)    2.442202  2.538162
               (3, 4)    2.538162  2.630836
               (4, 5)    2.630836  2.719298
...                           ...       ...
9      x3    7 (1, 8)    1.407902  1.417398
               (2, 9)    1.407953  1.422860
             8 (0, 8)    1.407896  1.417398
               (1, 9)    1.407902  1.422860
             9 (0, 9)    1.407896  1.422860

[1350 rows x 2 columns]

In [9]:
# mu_star calculation
mu_star_df = df[ishigami.__name__].groupby(level=[0,1]).mean()
mu_star_df.index.names = ['centre', 'param']
mu_star_df.unstack(level=1)

param,x1,x2,x3
centre,,,
0,2.656646,2.885317,2.959480
1,5.243696,2.068463,4.830351
2,4.864449,2.432892,5.084556
3,5.330994,2.254328,5.028727
4,2.554175,2.655503,2.704976
5,5.378425,2.651379,5.443588
6,2.075819,2.862646,2.280281
7,1.513483,2.958328,1.866293
8,4.072956,2.926255,4.365968


In [10]:
# overall mu (mean) of the unique evaluated function values over all stars points
mu_overall = df[ishigami.__name__].unique().mean()
mu_overall

3.2181307939105364

In [11]:
# overall var (variance) of the unique evaluated function values over all stars points
var_overall = df[ishigami.__name__].unique().var(ddof=1)
var_overall

2.625633323232234

In [12]:
# sectional covariogram calculation - content matches MATLAB code style!!
cov_section_all = cov_section(pair_df, mu_star_df)
cov_section_all.unstack(level=1)

param           x1        x2        x3
centre h                              
0      1  0.048396  2.130395  0.000085
       2  0.032084  1.415033  0.000045
       3  0.013009  0.545250  0.000009
       4 -0.008655 -0.447458 -0.000026
       5 -0.032701 -1.525340 -0.000059
...            ...       ...       ...
9      5 -0.033542 -1.506603 -0.000010
       6 -0.060431 -2.610690 -0.000016
       7 -0.089252 -3.710385 -0.000023
       8 -0.119703 -4.758991 -0.000031
       9 -0.151457 -5.710391 -0.000041

[90 rows x 3 columns]

In [13]:
# variogram calculation
variogram_value = variogram(pair_df)
variogram_value.unstack(level=0)

param,x1,x2,x3
h,,,
1,0.003723,0.157006,0.000009
2,0.014996,0.643154,0.000030
3,0.033887,1.467836,0.000055
4,0.060336,2.620288,0.000081
5,0.094159,4.067930,0.000110
6,0.135042,5.756585,0.000144
7,0.182547,7.612715,0.000191
8,0.236120,9.547591,0.000259
9,0.295096,11.463071,0.000365


In [14]:
# morris calculation
morris_values = morris_eq(pair_df)
morris_values[0].unstack(level=0)

param,x1,x2,x3
h,,,
1,0.085347,0.531766,0.002697
2,0.171708,1.090234,0.004997
3,0.258653,1.664431,0.007070
4,0.345745,2.242759,0.009086
5,0.432546,2.813301,0.011216
6,0.518617,3.364139,0.013629
7,0.603523,3.883672,0.016496
8,0.686833,4.360934,0.019987
9,0.768126,4.785892,0.024272


In [15]:
morris_values[1].unstack(level=0)

param,x1,x2,x3
h,,,
1,0.085347,0.531766,0.002697
2,0.171708,1.090234,0.004997
3,0.258653,1.664431,0.007070
4,0.345745,2.242759,0.009086
5,0.432546,2.813301,0.011216
6,0.518617,3.364139,0.013629
7,0.603523,3.883672,0.016496
8,0.686833,4.360934,0.019987
9,0.768126,4.785892,0.024272


In [16]:
# overall covariogram calculation
covariogram_value = covariogram(pair_df, mu_overall)
covariogram_value.unstack(level=0)

param,x1,x2,x3
h,,,
1,2.532258,2.555273,2.193087
2,2.517009,1.886561,2.192966
3,2.498010,1.042883,2.192949
4,2.475428,0.054558,2.193032
5,2.449472,-1.041330,2.193213
6,2.420388,-2.202352,2.193492
7,2.388463,-3.382391,2.193875
8,2.354013,-4.533451,2.194370
9,2.317387,-5.607577,2.194989


In [17]:
# expected value of the overall covariogram calculation
e_covariogram_value = e_covariogram(cov_section_all)
e_covariogram_value.unstack(level=0)

param,x1,x2,x3
h,,,
1,0.048206,2.064049,0.000049
2,0.031950,1.368743,0.000026
3,0.012943,0.524264,0.000005
4,-0.008639,-0.438263,-0.000015
5,-0.032586,-1.481585,-0.000034
6,-0.058656,-2.563760,-0.000055
7,-0.086574,-3.639797,-0.000078
8,-0.116039,-4.663456,-0.000104
9,-0.146724,-5.589107,-0.000136


In [125]:
# sobol calculation
sobol_value = sobol_eq(variogram_value, e_covariogram_value, var_overall)

param,x1,x2,x3
0,0.019777,0.845912,0.000022


In [19]:
# IVARS calculation
ivars_values = [0.1, 0.3, 0.5]
ivars_df = pd.DataFrame.from_dict({scale: variogram_value.groupby(level=0).apply(ivars, scale=scale, delta_h=delta_h) \
                      for scale in ivars_values}, 'index')
ivars_df

,x1,x2,x3
0.1,0.000186,0.007850,4.686609e-07
0.3,0.003566,0.153408,6.656532e-06
0.5,0.016002,0.692225,2.297963e-05


In [132]:
# bootstrapping to get CIs

# create result dataframes/series if bootstrapping is chosen to be done
result_bs_variogram = pd.DataFrame()
result_bs_sobol = pd.DataFrame()
result_bs_ivars_df = pd.DataFrame()

for iter in range(0, 10):
    ## specify random sequence by sampling with replacement
    bootstrap_rand = np.random.choice(list(range(0,10)), size=len(range(0,10)), replace=True).tolist()
    bootstrapped_pairdf = pd.concat([pair_df.loc[idx[i, :, :, :], :] for i in bootstrap_rand])
    bootstrapped_df     = pd.concat([df.loc[idx[i, :, :], :] for i in bootstrap_rand])
    #display(bootstrapped_pairdf)
    #display(bootstrap_rand)

    ## calculating sectional covariograms
    bootstrapped_cov_section_all = pd.concat([cov_section_all.loc[idx[i, :]] for i in bootstrap_rand])
    #display('sectional variogram:')
    #display(bootstrapped_cov_section_all)
    #display(bootstrap_rand)

    ## calculating variogram, ecovariogram, variance, mean, Sobol, and IVARS values
    bootstrapped_variogram = variogram(bootstrapped_pairdf)
    #display('variogram:')
    #display(bootstrapped_variogram.unstack(level=0))

    bootstrapped_ecovariogram = e_covariogram(bootstrapped_cov_section_all)
    #display('E(covariogram):')
    #display(bootstrapped_ecovariogram.unstack(level=0))

    bootstrapped_var = bootstrapped_df[ishigami.__name__].unique().var(ddof=1)
    #display('variance:', bootstrapped_var)

    bootstrapped_sobol = sobol_eq(bootstrapped_variogram, bootstrapped_ecovariogram, bootstrapped_var)
    #display('sobol:', bootstrapped_sobol)

    ivars_values = [0.1, 0.3, 0.5]
    delta_h = 0.1
    bootstrapped_ivars_df = pd.DataFrame.from_dict({scale: bootstrapped_variogram.groupby(level=0).apply(ivars, scale=scale, delta_h=delta_h) \
                                                    for scale in ivars_values}, 'index')
    #display('ivars:', boostrapped_ivars_df)
    
    # unstack variogram
    bootstrapped_variogram_df = bootstrapped_variogram.unstack(level=0)
    
    # transpose sobol values for stacking of results
    bootstrapped_sobol_df = bootstrapped_sobol.to_frame().transpose()
    
    # attatch new results to previous results (order does not matter here)
    result_bs_variogram = pd.concat([bootstrapped_variogram_df, result_bs_variogram])
    result_bs_sobol = pd.concat([bootstrapped_sobol_df, result_bs_sobol])
    result_bs_ivars_df = pd.concat([bootstrapped_ivars_df, result_bs_ivars_df])
    

    
    

#all these functions must be put in a for loop to derive the CIs at the end
# use pd.concat to put together all the matrices and calculate CIs - bootstrapping is done!

In [69]:
# calculate upper and lower confidence interval limits of the ivars values
param_names = ['x1', 'x2', 'x3']
ivars_scales = [0.1, 0.3, 0.5]
ivars_low = pd.DataFrame()
ivars_upp = pd.DataFrame()
for scale in ivars_scales:
    ivars_low = pd.concat([ivars_low, result_bs_ivars_df.loc[scale].quantile((1-0.9)/2).rename(scale).to_frame()], axis=1)
    ivars_upp = pd.concat([ivars_upp, result_bs_ivars_df.loc[scale].quantile(1-((1-0.9)/2)).rename(scale).to_frame()], axis=1)

ivars_low = ivars_low.transpose()
ivars_upp = ivars_upp.transpose()
display(ivars_low)
display(ivars_upp)

,x1,x2,x3
0.1,0.000183,0.007623,3.820809e-07
0.3,0.003509,0.148275,5.389820e-06
0.5,0.015740,0.666384,1.848930e-05


,x1,x2,x3
0.1,0.000187,0.007961,6.200276e-07
0.3,0.003591,0.155904,8.785493e-06
0.5,0.016117,0.704791,3.026205e-05


In [189]:
variogram_low = pd.DataFrame()
variogram_upp = pd.DataFrame()
for h in np.unique(result_bs_variogram.index.values).tolist():
    variogram_low = pd.concat([variogram_low, result_bs_variogram.loc[h].quantile((1-0.9)/2).rename(h).to_frame()], axis=1)
    variogram_upp = pd.concat([variogram_upp, result_bs_variogram.loc[h].quantile(1-((1-0.9)/2)).rename(h).to_frame()], axis=1)
    
variogram_low = variogram_low.transpose()
variogram_upp = variogram_upp.transpose()

variogram_low.index.names = ['h']
variogram_upp.index.names = ['h']

display(variogram_low)
display(variogram_upp)

param,x1,x2,x3
h,,,
1,0.003682,0.154367,0.000007
2,0.014833,0.631088,0.000021
3,0.033517,1.437755,0.000038
4,0.059678,2.562651,0.000057
5,0.093131,3.973285,0.000077
6,0.133567,5.616687,0.000101
7,0.180554,7.421595,0.000133
8,0.233541,9.302434,0.000181
9,0.291873,11.164804,0.000256


param,x1,x2,x3
h,,,
1,0.003784,0.160233,0.000013
2,0.015243,0.657912,0.000041
3,0.034447,1.504628,0.000076
4,0.061337,2.690783,0.000113
5,0.095723,4.183688,0.000153
6,0.137289,5.927690,0.000202
7,0.185588,7.846468,0.000267
8,0.240057,9.847436,0.000362
9,0.300019,11.827873,0.000508


In [143]:
result_bs_sobol.quantile((1-0.9)/2).rename('').to_frame().transpose()

param,x1,x2,x3
,0.01827,0.779509,0.000015


In [141]:
#result_bs_sobol.sort_values('x1')

In [229]:
def factor_ranking(factors):
    # gather indices for sorting factor
    temp = np.argsort(factors)[::-1]
    # create an array the same shape and type as temp
    ranks = np.empty_like(temp)
    # rank factors
    ranks[temp] = np.arange(len(factors))

    return ranks
    

In [230]:
a = np.array([9, 5, 7, 6])
factor_ranking(a)

array([0, 3, 1, 2], dtype=int64)

In [224]:
temp = a.argsort()[::-1]
temp

array([0, 2, 3, 1], dtype=int64)

In [225]:
ranks = np.empty_like(temp)

In [226]:
ranks[temp] = np.arange(len(a))
ranks

array([0, 3, 1, 2], dtype=int64)